In [21]:
# new data structure for STOPWATCHER
# build up tables BY VEHICLE - a history of observations of each vehicle
# then can go and reconstruct unique history for each VEHICLE, based on last observeration before it STOPPED APPROACHING each stop in the _stoplist_

In [1]:
from lib.reportcard_helpers import *

In [2]:
source = 'nj'
route = 87

In [3]:
(conn, db) = db_setup(route)

In [4]:
# only get buses APPROACHING A STOP
arrival_query = ('SELECT * FROM stop_predictions \
                WHERE (rd = %s AND pt = "APPROACHING") \
                ORDER BY timestamp;' % route)

In [5]:
df = pd.read_sql_query(arrival_query, conn)

In [6]:
df = timestamp_fix(df)

In [7]:
# 1 group all arrival predictions by unique vehicle-stop combinations
df.groupby(['v','stop_id'])

# 2 gets the keys for unique vehicle-stop combinations
# a list of tuples
# [(u'5403', u'20640'), (u'5704', u'20933'), .....

keys = df.groupby(['v','stop_id']).groups.keys()


In [8]:
# history data structure

#     append it to the value of the right dict entry
    
# {
#  5403_20640: [row1, row15, ...]
#  5704_20933: [row2, row37, ...]    
# }
    
# assign every row in df to a dict

from collections import defaultdict
history = defaultdict(list)

for index,row in df.iterrows():
    key = row['v'] + '_' + row['stop_id']
    history[key].append(row)


In [9]:
print history

defaultdict(<type 'list'>, {u'5723_20635': [pkey                               47399
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                      StopPrediction
pt                           APPROACHING
rd                                    87
rn                                    87
scheduled                          false
stop_id                            20635
stop_name    CENTRAL AVE + MANHATTAN AVE
v                                   5723
timestamp            2018-06-06 15:21:05
Name: 2018-06-06 15:21:05, dtype: object], u'5739_21124': [pkey                               39568
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                      StopPrediction
pt                           APPROACHING
rd                                  

In [ ]:
#
# PROBLEM WITH BELOW IS THAT IT DOESNT WORK WHEN WE HAVE A FULL HISTORY
# BECAUSE IT DOESNT DETECT EVENTS? e.g. VEHICLE-STOP COMBINATIONS ARE NOT UNIQUE FOR MORE THAN A FEW HOURS
#

In [17]:
# sort them and slice them

keepers = defaultdict(list)
discards = defaultdict(list)

# iterate over the dict
for key_copy, arrivals in history.iteritems():
    
    # sort each arrival list
    arrivals.sort(key=lambda x: x.timestamp)
    
    # put last row in keepers
    keepers[key_copy].append(arrivals[-1])
  
    # put everything else in discards
    discards[key_copy].append(arrivals[:-1])
    
#     for arrival in arrivals:
#         print arrival['timestamp']

#     for arrival in arrivals:        
#         # To sort the list in place...
#         

#     try:
#         # put last row in keepers
#         keepers[key_copy].append(arrivals[-1])
#     except:
#         keepers[key_copy].append(arrivals[0])
    
#     # put the rest in discards
#     keepers[key_copy].append(arrivals[:-1])


In [16]:
print keepers

defaultdict(<type 'list'>, {u'5739_21124': [pkey                               39568
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                      StopPrediction
pt                           APPROACHING
rd                                    87
rn                                    87
scheduled                          false
stop_id                            21124
stop_name       SUMMIT AVE + PAVONIA AVE
v                                   5739
timestamp            2018-06-06 14:53:07
Name: 2018-06-06 14:53:07, dtype: object], u'5395_20944': [pkey                                 14953
cars                                      
consist                                   
fd             87 GATES AVE VIA JOURNAL SQ
m                                        1
name                        StopPrediction
pt                             APPROACHING
rd                    

In [18]:
print discards

defaultdict(<type 'list'>, {u'5739_21124': [[]], u'5395_20944': [[pkey                                 14653
cars                                      
consist                                   
fd             87 GATES AVE VIA JOURNAL SQ
m                                        1
name                        StopPrediction
pt                             APPROACHING
rd                                      87
rn                                      87
scheduled                            false
stop_id                              20944
stop_name    MONTICELLO AVE + HARRISON AVE
v                                     5395
timestamp              2018-06-04 16:40:02
Name: 2018-06-04 16:40:02, dtype: object]], u'5397_20613': [[]], u'5900_32216': [[]], u'5242_20717': [[pkey                               11237
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                    